# Les modèles pour l’apprentissage supervisé

Pour mémoire, les modèles pour l’apprentissage supervisé sont classés en fonction de la tâche à réaliser :
- La **régression**, quand la prévision porte sur une valeur continue comme le salaire d’un·e étudiant·e en fin de formation ;
- ou la **classification** lorsqu’il s’agit de prédire l’appartenance à une classe.

Certains modèles sont réservés à l’une ou l’autre de ces tâches quand les autres peuvent servir aux deux. Dans ce calepin, nous parcourons les plus populaires sans pour autant rentrer dans les détails.

1. **Les modèles pour la régression :**
    - [La régression linéaire](#La-régression-linéaire)
    - [La régression polynomiale](#La-régression-polynomiale)
2. **Les modèles pour la classification :**
    - [Les K plus proches voisins](#Les-K-plus-proches-voisins)
    - [La classification naïve bayésienne](#La-classification-naïve-bayésienne)
    - [La régression logistique](#La-régression-logistique)
3. **Les modèles mixtes :**
    - [Les séparateurs à vaste marge](#Les-séparateurs-à-vaste-marge) (SVM)
    - [Les arbres de décision](#Les-arbres-de-décision)
    - [Les forêts aléatoires](#Les-forêts-aléatoires)

Avant de commencer, chargeons les librairies nécessaires et constituons des ensembles de données pour les différentes tâches à traiter :

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_selector as col_selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

###################
# Regression task #
###################

# for deterministic purposes
np.random.seed(0)

# hundred points
X = np.random.rand(100, 1)

# polynomial function: y = 2 - 3x + 3x^2 + 5x^3
y = 2 - 3 * X + 3 * X ** 2 + 5 * X ** 3 + np.random.rand(100, 1)

#######################
# Classification task #
#######################

# penguin census
df = pd.read_csv("./data/penguin-census.csv").dropna()

# features & target
target = "species"
features = ["flipper_length_mm", "body_mass_g", "sex"]

# dataset
X_b = df[features]
y_b = df[target]

# selectors
num_col_selector = col_selector(dtype_exclude=object)
cat_col_selector = col_selector(dtype_include=object)

# num & cat cols
num_cols = num_col_selector(X_b)
cat_cols = cat_col_selector(X_b)

# preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num-transformers", StandardScaler(), num_cols),
        ("cat-transformers", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

## Avant-propos

Les modèles d’apprentissage de *Scikit-Learn*, programmés en différentes classes, sont conçus autour du même esprit de cohésion. Chaque classe expose notamment les mêmes méthodes suivantes :
- `.fit()` pour ajuster le modèle sur des données distribuées dans deux paramètres `x` et `y` ;
- `.predict()` pour effectuer des prédictions sur des données ;
- `.score()` pour obtenir une évaluation des prédictions obtenues.

Nous nous limitons ici aux aspects essentiels des modèles, en leur laissant un maximum de degrés de liberté quand ils devraient être régularisés pour éviter les effets de sous-ajustement (*underfitting*) ou de sur-ajustement (*overfitting*). Les scores obtenus, souvent très satisfaisants, ne doivent pas égarer : d’une part les modèles sont entraînés sur des jeux de donnés complets quand il aurait fallu les tester et les valider sur autant de partitions ; d’autre part, ils ne sont jamais évalués relativement à des prédictions sur des données nouvelles, interdisant de fait de se rendre compte à quel point ils se généraliseraient mal.

Pour comprendre en détail le fonctionnement de chaque modèle, nous renvoyons à [la documentation officielle](https://scikit-learn.org/stable/).

## La régression linéaire

Sans doute la méthode la plus populaire en apprentissage supervisé, la régression linéaire consiste à trouver la fonction affine $y$ d’une variable explicative $x$. Elle permet de comprendre rapidement la relation entre deux variables. Pour autant, sa sensibilité aux données aberrantes et aux *outliers* peut l’empêcher d’observer une corrélation ; elle se révèle alors une hypothèse trop simple pour les données. On parle alors de sous-ajustement (*underfitting*).

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
_ = model.fit(X, y)

Une fois le modèle entraîné, il est possible d’accéder au coefficient directeur de la droite et à son ordonnée à l’origine grâce aux attributs `coef_` et `intercept_` :

In [ ]:
print(model.coef_, model.intercept_)

Tout comme en connaître la précision, ici grâce au coefficient de détermination $R^2$ :

In [ ]:
model.score(X, y)

L’interprétation des résultats d’une régression linéaire est extrêmement simple et généralement immédiate :

In [ ]:
_ = sns.regplot(x=X, y=y)

## La régression polynomiale

Le modèle de régression linéaire par ajustement affine ne s’est pas révélé une bonne hypothèse pour nos données d’entrée. Pour rappel, nous avions utilisé un polynôme de degré 3 pour générer les coordonnées en $y$.

Lorsque, en visualisant les données, on observe une relation non-linéaire, l’hypothèse est qu’il doit exister d’autres caractéristiques pour l’expliquer que simplement $X$. *Scikit-Learn* met à disposition une classe `PolynomialFeatures` pour les ajouter à $X$ en fonction du polynôme considéré dans un paramètre `degree` :

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

model = make_pipeline(
    PolynomialFeatures(degree=3),
    LinearRegression(),
)
model.fit(X, y)
y_pred = model.predict(X)

Le modèle s’ajuste bien mieux aux données que la précédente régression linéaire, ce qui se traduit immédiatement dans une augmentation de son *$R^2$ score* :

In [ ]:
ax = sns.scatterplot(x=X[:, 0], y=y[:, 0])
ax.plot(sorted(X), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X, y):.2f}")

Il serait tentant de chercher à améliorer le *$R^2$ score* en augmentant le degré du polynôme, mais le risque serait d’ajuster trop bien le modèle (*overfitting*) aux données d’entraînement qu’il ne parviendrait plus à obtenir de bonnes prédictions sur des données nouvelles :

In [ ]:
model = make_pipeline(
    PolynomialFeatures(degree=15),
    LinearRegression(),
)
model.fit(X, y)

y_pred = model.predict(X)

ax = sns.scatterplot(x=X[:, 0], y=y[:, 0])
ax.plot(sorted(X), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X, y):.2f}")

La courbe semble plus fortement attirée par les données avec un polynôme de très haut degré, ce qui, en quelque sorte, la rend dépendante vis-à-vis d’elles. Il faut s’attendre à voir le *$R^2$ score* diminuer sensiblement sur des données nouvelles :

In [ ]:
np.random.seed(1)

X_prime = np.random.rand(100, 1)
y_prime = 2 - 3 * X_prime + 3 * X_prime ** 2 + 5 * X_prime ** 3 + np.random.rand(100, 1)

y_pred = model.predict(X_prime)

ax = sns.scatterplot(x=X_prime[:, 0], y=y_prime[:, 0])
ax.plot(sorted(X_prime), sorted(y_pred))
_ = ax.set_title(f"R2 score = {model.score(X_prime, y_prime):.2f}")

## Les K plus proches voisins

Le modèle de recherche des $k$ plus proches voisins (*K-Nearest Neighbors*) consiste à rechercher, pour toute nouvelle entrée, les $k$ entrées les plus proches dans la base d’apprentissage. Il est par ailleurs possible de préciser $k$ grâce à un paramètre `n_neighbors`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = make_pipeline(preprocessor, KNeighborsClassifier(n_neighbors=3))
_ = model.fit(X_b, y_b)

Parmi les attributs intéressants de la classe, citons `classes_` et `feature_names_in_` qui permettent de garder une trace des différentes modalités ainsi que des variables explicatives utilisées dans le modèle :

In [ ]:
print(
    f"Modalités : {model.classes_}",
    f"Variables explicatives : {model.feature_names_in_}",
    sep="\n"
)

Le score obtenu est le taux de prédictions exactes sur l’ensemble du jeu d’apprentissage :

In [ ]:
model.score(X_b, y_b)

## La classification naïve bayésienne

La classification naïve bayésienne repose sur l’idée qu’il existe une forte indépendance entre les variables explicatives d’un jeu de données et, partant, qu’elles contribuent individuellement autant les unes que les autres à la probabilité qu’une observation appartienne à une modalité. La contraposée implique qu’un classifieur naïf bayésien ne fournira pas de bons résultats si les variables sont effectivement corrélées entre elles.

S’il existe plusieurs types de classifieurs, nous ne verrons que l’algorithme appelé *Gaussian Naïve Bayes* :

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = make_pipeline(preprocessor, GaussianNB())
model.fit(X_b, y_b)
model.score(X_b, y_b)

Si ce modèle est rapide à mettre en place, il est en revanche moins performant que d’autres algorithmes que les arbres de décision ou les séparateurs à vaste marge. Il a toutefois l’avantage de pouvoir être entraîné sur un faible volume de données.

## La régression logistique

L’algorithme de régression logistique est très souvent employé dans les tâches de classification car il peut estimer rapidement la probabilité qu’une observation appartienne à une modalité particulière. Un paramètre `class_weight` permet de transmettre un dictionnaire de poids pour les modalités afin de pondérer la classification, ce qui peut se révéler utile pour corriger un biais de représentativité. Dans le cas contraire, toutes les étiquettes sont réputées avoir un poids de 1. Il est également possible de modifier l’algorithme utilisé par défaut pour l’optimisation de la régression avec un paramètre `solver`. Notons également un paramètre `max_iter` fixé par défaut à 100, qu’il peut être utilse d’augmenter afin de faire disparaître un avertissement `ConvergenceWarning` :

In [ ]:
from sklearn.linear_model import LogisticRegression

weights = {
    "Adelie": .4,
    "Gentoo": .2,
    "Chinstrap": .4
}

model = make_pipeline(
    preprocessor,
    LogisticRegression(
        max_iter=100,
        class_weight=weights,
        solver="liblinear"
    )
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

## Les séparateurs à vaste marge

Les séparateurs à vaste marge (SVM), autrement appelés machines à vecteur de support (*Support Vector Machines*) sont réputées pour leur excellence à traiter une multitude de tâches : régression ou classification (que les données soient linéaires ou non) mais aussi de la détection de données aberrantes. Ils sont par ailleurs très sensibles aux différences entre les échelles des variables, aussi il est fortement recommandé de normaliser toutes les données avant la phrase d’entraînement.

Pour les tâches de classification, la classe à utiliser est `LinearSVC` :

In [ ]:
from sklearn.svm import LinearSVC

model = make_pipeline(
    preprocessor,
    LinearSVC()
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

Et pour les tâches de régression, `LinearSVR` :

In [ ]:
from sklearn.svm import LinearSVR

model = LinearSVR()
model.fit(X, y[:, 0])
model.score(X, y)

### SVM pour la classification

#### Classification à marge rigide ou souple

Pour opérer la classification, les SVM vont effectuer un découpage linéaire des données en veillant à respecter une distance égale avec les observations les plus proches, appelée marge (paramètre `C`). Cette marge peut être rigide, imposant qu’aucune observation ne se trouve à l’intérieur, ou souple, autorisant des empiètements de marge. En théorie, il est préférable d’avoir peu d’empiètements de marge, mais une classification à marge rigide ne pourra être mise en place que si les observations sont linéairement séparables par une frontière nette, c’est-à-dire sans aucune observation en dehors de son espace. Pour cette raison, on optera dans la pratique plutôt pour un modèle permissif qui se généralisera sans doute mieux.

Afin de représenter la séparation, prenons une extraction de nos données avec seulement deux modalités (*Adelie* et *Gentoo*), et entraînons un classifieur SVM à marge souple :

In [ ]:
# dataset with two features + target
data = df.filter(["flipper_length_mm", "body_mass_g", "species"])
data = data[data["species"] != "Chinstrap"]
X_c = data.drop(columns="species")
y_c = data["species"]

# standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_c)

# soft margin SVM
model = LinearSVC(C=1, loss="hinge")
_ = model.fit(X_scaled, y_c)

Le modèle fournit le coefficient directeur $a$, composé ici de deux valeurs, une pour chaque variable explicative, ainsi que l’ordonnée à l’origine $b$ qui vont permettre de calculer les coordonnées :

In [ ]:
# slope (two weights: one for each feature) and intercept
a = model.coef_[0]
b = model.intercept_[0]

L’équation générale sous forme vectorielle de la limite de décision vaut :

$$\left < a, x \right > + b = 0$$

Si nous la convertissons sous forme linéaire, nous obtenons :

$$
    \left[ {\begin{array}{cc}
        a_1 & a_2
    \end{array} } \right] \times
    \left[ {\begin{array}{cc}
        x_1 \\
        x_2 \\
    \end{array} } \right] + b = 0
$$

$$a_2x_2 = -a_1x_1 -b$$

$$x_2 = - \frac{a_1}{a_2} x_1 - \frac{b}{a_2}$$

Dans cette dernière forme, $a_1$ et $a_2$ sont les coefficients directeurs, $x_1$ sont les coordonnées sur l’axe des abscisses et $x_2$ les coordonnées sur l’axe des ordonnées.

In [ ]:
# 2D-line equation
x_coords = np.linspace(-1, 1)
y_coords = -(a[0] / a[1]) * x_coords - b / a[1]

Il ne reste plus qu’à afficher le graphique :

In [ ]:
ax = plt.subplots(figsize=(10,8))

ax = sns.lineplot(x=x_coords, y=y_coords, color="fuchsia")
ax = sns.scatterplot(x=X_scaled[:, 0], y=X_scaled[:, 1], hue=y_c)

ax.set(xlabel="Body mass (g)", ylabel="Flipper length (mm)", title="Frontière de décision d’un SVM à marge souple (C=1)")

sns.despine()

plt.show()

#### Classification pour des données non linéaires

Dans l’exemple précédent, les données étaient, à l’exception de quelques observations qui empiétaient dans l’espace de l’autre modalité, linéairement séparables, et la classe `LinearSVC` était indiquée pour résoudre rapidement la tâche d’apprentissage. Si nous reprenons le jeu de données initial avec les trois modalités, la séparation entre les données devient moins évidente :

In [ ]:
_ = sns.scatterplot(data=X_b, x="body_mass_g", y="flipper_length_mm", hue=y_b)

##### Un SVM à noyau polynomial

Rien n’interdit de recourir à un SVM linéaire mais, dans le cas de notre exemple, nous voudrons sans doute rigidifier les marges pour éviter le risque de sous-ajustement et les erreurs de généralisation qu’il entraînerait. Comment alors ajuster mieux la frontière de décision ? La solution est d’utiliser un polynôme :

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

model = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=3),
    LinearSVC(C=10, max_iter=50000)
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

Les résultats sont meilleurs mais au détriment de temps de calculs nécessairement plus longs : l’ajout de variables polynomiales nuit d’autant plus à l’équilibre de l’ensemble que nous aurions besoin d’un polynôme à degré très élevé pour s’adapter à des données complexes.

Une astuce consiste à utiliser plutôt la classe `SVC` en lui associant un noyau polynomial avec le paramètre `kernel` et en contrôlant l’influence des polynômes de degré élevé avec un paramètre `coef0` :

In [ ]:
from sklearn.svm import SVC

model = make_pipeline(
    preprocessor,
    SVC(kernel="poly", degree=6, C=10, coef0=1)
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

##### Un SVM à noyau radial gaussien

Notre jeu de données ne se prête pas plus à une séparation avec une droite qu’avec une fonction polynomiale. Heureusement, une autre méthode lui correspond mieux. L’idée consiste à calculer une fonction de similarité pour mesurer la ressemblance entre une observation et un point de repère. Pour chaque observation, une nouvelle variable calculée : le temps de traitement d’un vaste *dataset* risque d’exploser.

Comme dans le cas du noyau polynomial, il existe une astuce pour obtenir le résultat sans ajouter dans les faits les variables de similarité : fixer le paramètre `kernel` à `rbf`. Le point clé consiste alors à régler le paramètre $\gamma$ qui définit la forme de la frontière de décision : de lisse avec une valeur faible, elle devient plus irrégulière avec une valeur forte.

In [ ]:
from sklearn.svm import SVC

model = make_pipeline(
    preprocessor,
    SVC(kernel="rbf", gamma=10, C=10)
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

##### `SVC` ou `LinearSVC` ?

La classe `LinearSVC` ne repose pas sur la même librairie que `SVC` et sera nettement plus rapide lorsque les modalités sont linéairement séparables. Dans les autres cas, `SVC` donnera de meilleurs résultats : il s’adapte mieux aux données complexes et intègre *l’astuce du noyau*. Toutefois, dès lors que le volume du jeu d’entraînement devient important, on observe un ralentissement notable de l’apprentissage dû à la complexité de son algorithme. Cette restriction ne concerne pas le nombre de variables explicatives, surtout si elles sont dites creuses (avec peu de valeurs non nulles).

### SVM pour la régression

Pour résoudre des tâches de classification, les modèles SVM s’efforcent de séparer les données en délimitant une frontière la plus large possible tout en évitant les empiètements à l’intérieur. Dans le cas des tâches de régression, c’est l’inverse : les modèles SVM tentent le mieux possible de contenir les observations à l’intérieur du chemin qui la délimite. Le paramètre `epsilon` gouverne la largeur du chemin. Le modèle est alors réputé être insensible à $\epsilon$ près, car rajouter des observations d’entraînement ne l’affecte pas.

La classe `LinearSVR` est prévue pour des applications impliquant des données linéaires :

In [ ]:
from sklearn.svm import LinearSVR

model = LinearSVR(epsilon=0.5, C=10, max_iter=2000)
model.fit(X, y[:, 0])
model.score(X, y)

La classe `SVR` est quant à elle parfaite pour traiter des données non linéaires :

In [ ]:
from sklearn.svm import SVR

model = SVR(kernel="poly", degree=4, epsilon=0.5, C=1)
model.fit(X, y[:, 0])
model.score(X, y)

## Les arbres de décision

Briques fondamentales des forêts aléatoires, les arbres de décision sont capables de s’adapter à des jeux de données complexes pour effectuer tout à la fois des tâches de classification et de régression. En les laissant libre pendant la phase d’apprentissage, ils arborent fièrement des résultats au-delà de toute espérance :

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = make_pipeline(
    preprocessor,
    DecisionTreeClassifier()
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

### Interprétation des résultats

La promesse d’une précision élevée à l’entraînement cache toutefois une tendance au sur-ajustement des arbres de décision. S’ils ne sont pas régularisés en amont, ils provoqueront des erreurs de généralisation. L’intuition serait alors de jouer sur les hyperparamètres, mais ils font partie des modèles non paramétriques et n’en exposent pas avant l’apprentissage. Aussi, la seule manière de limiter leur degré de liberté est de contrôler quelques paramères de la classe :
- `max_depth`, pour contraindre la profondeur de l’arbre ;
- `min_samples_split`, pour indiquer le nombre minimum d’observations à considérer avant que le nœud se divise ;
- `min_samples_leaf`, pour forcer les nœuds terminaux à contenir un minimum d’observations ;
- `max_features`, pour renseigner le nombre de variables explicatives considérées à chaque nœud.

Le score de précision peut diminuer fortement, au bénéfice toutefois d’une meilleure généralisation.

Un autre avantage des arbres de décision, et non des moindres, est qu’ils nécessitent très peu de préparation des données, comme dans l’exemple ci-dessous où la variable *sex* a simplement été recodée en 0 et 1 :

In [ ]:
X_c = X_b.replace({"male": 0, "female": 1})

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_c, y_b)
model.score(X_c, y_b)

### Algorithme de décision

Le modèle des arbres de décisions repose sur l’algorithme CART (*Classification And Regression Trees*) qui ne produit que des arbres binaires, comme dans l’organigramme ci-dessous qui reprend le dernier modèle où la profondeur maximale est fixée à 3 niveaux :

![Modélisation des décisions de l’algorithme](./images/penguin-tree.png)

Au moment de l’entraînement, le modèle a lui-même évalué laquelle des trois variables explicatives entre *flipper_length_mm*, *body_mass_g* et *sex* était la plus discriminante pour effectuer la première division. Il calcule ensuite une frontière de décision et suppose une modalité qui repose sur l’indice de diversité de Gini (ou indice d’impureté) :

$$G_i = 1 - \overset{n}{\underset{k=1}{\sum}} P_{i,k^2} $$

Concrètement, tout en haut de l’arbre, l’algorithme se demande si les nageoires d’un manchot sont inférieures ou égales à 206,5 mm. Si tel est le cas, il émet l’hypothèse qu’il s’agit d’un manchot Adélie et passe la main au nœud inférieur situé à l’embranchement de gauche. Ici, la variable explicative est la même, mais la frontière est abaissée à 192,50 mm. Si l’on veut calculer l’indice d’impureté à ce niveau :

$$G = 1 - \left(\frac{144}{208}\right)^2 - \left(\frac{63}{208}\right)^2 - \left(\frac{1}{208}\right)^2$$

In [ ]:
gini = 1 - (144/208) ** 2 - (63/208) ** 2 - (1/208) ** 2
print(f"Indice d’impureté de Gini : {gini:.3f}")

### Estimation des probabilités

Pour en terminer sur les arbres de décision, notons la présence d’une méthode `.predict_proba()` qui renvoie l’estimation des probabilités d’appartenance à une modalité ou l’autre pour une observation. Prenons un manchot mâle de 3,950 kg et dont les nageoires mesurent 201 mm et suivons l’arbre de décision :
1. Nageoires inférieures ou égales à 206,5 mm ? **Vrai**
2. Nageoires inférieures ou égales à 192,5 mm ? **Faux**
3. Masse corporelle inférieure ou égale à 4,175 kg ? **Vrai**

L’algorithme devrait répondre qu’il s’agit d’un manchot à jugulaire (*Chinstrap*) avec une probabilité de 55 % (38 sur 69 observations) :

In [ ]:
penguin = pd.DataFrame({
    "flipper_length_mm": [201],
    "body_mass_g": [3950],
    "sex": [0]
})
print(
    f"Modalités : { ' '.join(model.classes_) }",
    f"Probabilités : { model.predict_proba(penguin)[0] }",
    f"Espèce : { model.predict(penguin)[0] }",
    sep="\n"
)

### Arbres de décision pour des tâches de régression

Les arbres de décision s’adaptent tout aussi bien aux tâches de régression. Au lieu de prédire une modalité, il prédit une valeur et décide de l’embranchement à suivre en prenant comme fonction de coût l’erreur quadratique moyenne (MSE) :

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X, y)
model.score(X, y)

Comme lorsqu’ils sont mobilisés pour des tâches de classification, les modèles avec arbres de décision ont une nette tendance au sur-ajustement. Le score $R^2$ très élévé tend à le suggérer. Pour le régulariser et améliorer ainsi ses performances au moment de la confrontation avec des données inédites, il faut chercher les bons ajustements des hyperparamètres.

Construisons un modèle qui se généralisera mieux :

In [ ]:
model_b = DecisionTreeRegressor(min_samples_leaf=10)
model_b.fit(X, y)
model_b.score(X, y)

Effectuons à présent des prédictions à l’aide des deux modèles :

In [ ]:
y_pred_over = model.predict(X)
y_pred = model_b.predict(X)

Et affichons en dernier lieu les graphiques afin de comparer les courbes :

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))

# overfitted model
sns.scatterplot(x=X[:, 0], y=y[:, 0], ax=ax1)
sns.lineplot(x=X[:, 0], y=y_pred_over, ax=ax1, color="lightsalmon")

# more reasonable model
sns.scatterplot(x=X[:, 0], y=y[:, 0], ax=ax2, color="orange")
sns.lineplot(x=X[:, 0], y=y_pred, ax=ax2, color="green")

ax1.set(title="Un modèle sur-ajusté ($R^2$ = 1)")
ax2.set(title="Un modèle plus convaincant ($R^2$ = 0,95)")

sns.despine()

plt.show()

## Les forêts aléatoires

Les forêts aléatoires, de l’anglais *random forest*, proposent un algorithme qui entraîne plusieurs arbres de décision en variant légèrement le jeu d’apprentissage. Elles reposent sur le principe de la sagesse des foules où l’erreur du groupe est réputée être inférieure à celle des individus qui le composent.

Dans *Scikit-Learn*, la classe `RandomForestClassifier` permet de mobiliser des forêts aléatoires pour des tâches de classification. La modalité retenue sera alors issue d’un vote majoritaire entre les différents arbres de décision. En plus des paramètres classiques de la classe `DecisionTreeClassifier`, `n_estimators` règle le nombre d’arbres sollicités et `n_jobs` indique le nombre de CPU à mobiliser (-1 pour tous les utiliser) :

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators=100, n_jobs=-1)
)
model.fit(X_b, y_b)
model.score(X_b, y_b)

Très pratique, soulignons la propriété `feature_importances_` pour connaître l’importance de chaque variable explicative dans l’estimation :

In [ ]:
model[1].feature_importances_

Et pour les tâches de régression, il faut appeler la classe `RandomForestRegressor` :

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    min_samples_leaf=10,
    n_estimators=100,
    n_jobs=-1
)
model.fit(X, y[:, 0])
model.score(X, y[:, 0])

La précision des résultats obtenus peut surprendre : elle est au mieux égale à celle d’un arbre de décision classique. Rappelons déjà que nos exemples apprennent depuis le jeu de données complet ; autrement, c’est aussi le signe que nos forêts aléatoires sont mal réglées. Il ne faut effectivement pas oublier que les abres de décisions ont une forte tendance au sur-ajustement et qu’un ensemble d’estimateurs défaillants ne saurait produire de prédiction efficace.